### Библиотеки

In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd 
from time import sleep

---

### Склеиваем url по нужной компании и датам

In [2]:
template_url = 'https://www.finam.ru/profile/moex-akcii/'
company = 'rosneft'
start_date = '2012-01-01'
end_date = '2021-08-14' 
page = '1'
url = template_url + company + '/news/?start-date=' + start_date + '&end-date=' + end_date + '&page=1'
base_url = template_url + company + '/news/' # чтобы запихнуть в requests
url

'https://www.finam.ru/profile/moex-akcii/rosneft/news/?start-date=2012-01-01&end-date=2021-08-14&page=1'

In [ ]:
#url = 'https://www.finam.ru/profile/moex-akcii/rosneft/news/?start-date=2012-01-01&end-date=2021-08-14&page=1'
#base_url = 
#start_date = url[65:75] 
#end_date = url[85:95] 
#page = url[-1] 

### Ищем максимальный номер страницы

In [3]:
resp = requests.get(base_url, 
                    params={'start-date': start_date,
                            'end-date': end_date,
                            'page': page})
soup = bs(resp.text)
pages = soup.find('div', {'class': 'pager'}).find_all('li')[-2].text.split('...')[-1]
pages

'123'

---

### Собираем со страницы заголовки, даты и ссылки на новость

In [4]:
news = []

for i in range(1,int(pages)+1):
    
    print('start page', i, end='\r')
    
    resp = requests.get(base_url, 
                    params={'page': i})
        
    soup = bs(resp.text)
    news_set = soup.find('tbody', {'class': 'news'}).find_all('tr')
    
    for item in news_set:
        date = item.find_all('td')[0].text
        preview = item.find_all('td')[1].text
        url = 'https://www.finam.ru' + item.find_all('td')[1].find('a').attrs['href']
        news.append({
            'preview_date': date,
            'preview': preview,
            'url': url,
            'company': company}) 
    sleep(1)

In [5]:
pd.DataFrame(news)

,preview_date,preview,url,company
0,13.08.2021,"Чистая прибыль ""Роснефти"" по РСБУ за 1 полугод...",https://www.finam.ru/analysis/newsitem/chistay...,rosneft
1,13.08.2021,"Совет директоров ""Роснефти"" в ближайшее время ...",https://www.finam.ru/analysis/newsitem/sovet-d...,rosneft
2,13.08.2021,Индексу МосБиржи не удается взять штурмом уров...,https://www.finam.ru/analysis/newsitem/indeksu...,rosneft
3,13.08.2021,"""Роснефть"" на конец 2 квартала строит 6 скважи...",https://www.finam.ru/analysis/newsitem/rosneft...,rosneft
4,13.08.2021,"""Роснефть"" увеличила добычу нефти и конденсата...",https://www.finam.ru/analysis/newsitem/rosneft...,rosneft
...,...,...,...,...
2438,27.01.2012,"""Газпром"" опустился в рейтинге публичных энерг...",https://www.finam.ru/analysis/newsitem6352F/,rosneft
2439,26.01.2012,"""Роснефть"" будет участвовать в развитии ХК ЦСК...",https://www.finam.ru/analysis/newsitem63489/,rosneft
2440,23.01.2012,FINAM расширяет возможности маржинальной торговли,https://www.finam.ru/analysis/newsitem631B9/,rosneft
2441,23.01.2012,"Доказанные запасы углеводородов ""Роснефти"" на ...",https://www.finam.ru/analysis/newsitem631AD/,rosneft


### Заходим на страничку новости по собранному url и скачиваем дату и время выхода новости 

In [6]:
for item in news:
        print('start news', news.index(item), end='\r')
        resp = requests.get(item['url'])
        soup = bs(resp.text)
        article_time = soup.find('div', {'class': 'sm lightgrey mb05 mt15'}).text[11:-18]
        article_date = soup.find('div', {'class': 'sm lightgrey mb05 mt15'}).text[:-24]
        item.update({'article_time': article_time,
                     'article_date': article_date})

In [7]:
pd.DataFrame(news)

,preview_date,preview,url,company,article_time,article_date
0,13.08.2021,"Чистая прибыль ""Роснефти"" по РСБУ за 1 полугод...",https://www.finam.ru/analysis/newsitem/chistay...,rosneft,20:55,13.08.2021
1,13.08.2021,"Совет директоров ""Роснефти"" в ближайшее время ...",https://www.finam.ru/analysis/newsitem/sovet-d...,rosneft,19:44,13.08.2021
2,13.08.2021,Индексу МосБиржи не удается взять штурмом уров...,https://www.finam.ru/analysis/newsitem/indeksu...,rosneft,15:11,13.08.2021
3,13.08.2021,"""Роснефть"" на конец 2 квартала строит 6 скважи...",https://www.finam.ru/analysis/newsitem/rosneft...,rosneft,10:57,13.08.2021
4,13.08.2021,"""Роснефть"" увеличила добычу нефти и конденсата...",https://www.finam.ru/analysis/newsitem/rosneft...,rosneft,10:50,13.08.2021
...,...,...,...,...,...,...
2438,27.01.2012,"""Газпром"" опустился в рейтинге публичных энерг...",https://www.finam.ru/analysis/newsitem6352F/,rosneft,13:12,27.01.2012
2439,26.01.2012,"""Роснефть"" будет участвовать в развитии ХК ЦСК...",https://www.finam.ru/analysis/newsitem63489/,rosneft,17:55,26.01.2012
2440,23.01.2012,FINAM расширяет возможности маржинальной торговли,https://www.finam.ru/analysis/newsitem631B9/,rosneft,13:15,23.01.2012
2441,23.01.2012,"Доказанные запасы углеводородов ""Роснефти"" на ...",https://www.finam.ru/analysis/newsitem631AD/,rosneft,12:22,23.01.2012


---

### Заходим на страничку новости и подтягиваем полный текст

In [8]:
item = news[5]
item

{'preview_date': '13.08.2021',
 'preview': 'Российские индексы торгуются волатильно, в плюсе бумаги HeadHunter',
 'url': 'https://www.finam.ru/analysis/newsitem/rossiiyskie-indeksy-torguyutsya-volatilno-v-plyuse-bumagi-headhunter-20210813-101812/',
 'company': 'rosneft',
 'article_time': '10:18',
 'article_date': '13.08.2021'}

In [9]:
resp = requests.get(item['url'])
soup = bs(resp.text)
soup

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns:og="http://ogp.me/ns#">
<head>
<title>Российские индексы торгуются волатильно, в плюсе бумаги HeadHunter - Новости компаний - Финам.ru</title>
<meta content='Российский рынок открылся сегодня в "красной" зоне, однако в моменте наблюдается волатильная динамика индексов. Индекс МосБиржи снижается в моменте на 0,08% до&amp;nbsp;...' name="Description"/>
<meta content="article" property="og:type"/>
<meta content="Российские индексы торгуются волатильно, в плюсе бумаги HeadHunter" property="og:title"/>
<meta content='Российский рынок открылся сегодня в "красной" зоне, однако в моменте наблюдается волатильная динамика индексов. Индекс МосБиржи снижается в моменте на 0,08% до&amp;nbsp;...' property="og:description"/>
<meta content="https://www.finam.ru/api/news/image?id=807521&amp;source=0&amp;nc=739&amp;target=fb" property="og:image"/>
<meta content="https://w

In [10]:
text_parts = soup.find('div', {'class': 'handmade mid f-newsitem-text'}).find_all('p')
text_parts

[<p>Российский рынок открылся сегодня в "красной" зоне, однако в моменте наблюдается волатильная динамика индексов. Индекс МосБиржи снижается в моменте на 0,08% до 3885,92 пункта, а индекс РТС пребывает в плюсе на 0,02%, торгуясь на отметке 1665,83 пункта.</p>,
 <p>Внешний фон на открытии основной торговой сессии в РФ является негативным. Азиатские индексы снижаются в связи с опасениями вокруг дальнейшего распространения COVID-19, а нефть дешевеет ввиду ожидаемого сокращения спроса на сырье в Китае. Фьючерсы на нефть марки Brent с поставкой в октябре опускаются на 0,6% до $70,88 за баррель, а Light – на 0,75% до $68,38 за баррель.</p>,
 <p>Пара EUR/USD находится на отметке 1,173 (+0,03%), американский доллар стоит 73,48 рубля (-0,15%), тогда как евро – 86,23 рубля (-0,18%).</p>,
 <p>"Роснефть" представила свои финансовые результаты по МСФО за 2 квартал 2021 года. Чистая прибыль компании увеличилась в 6 раз и составила 263 млрд рублей, а прибыль за 1 полугодие составила 431 млрд рублей,

In [11]:
text_parts[6].text

'Сегодня же появились сообщения о том, что "Яндекс" вышел из переговоров о приобретении сети магазинов "Азбука вкуса", что может быть связано с высокой оценкой актива. На утренних торгах рыночная капитализация "Яндекса" снижается на 0,2%.'

In [13]:
new_text_parts = []
for part in text_parts:
    new_text_parts.append(part.text)
new_text_parts
' '.join(new_text_parts)

'Российский рынок открылся сегодня в "красной" зоне, однако в моменте наблюдается волатильная динамика индексов. Индекс МосБиржи снижается в моменте на 0,08% до 3885,92 пункта, а индекс РТС пребывает в плюсе на 0,02%, торгуясь на отметке 1665,83 пункта. Внешний фон на открытии основной торговой сессии в РФ является негативным. Азиатские индексы снижаются в связи с опасениями вокруг дальнейшего распространения COVID-19, а нефть дешевеет ввиду ожидаемого сокращения спроса на сырье в Китае. Фьючерсы на нефть марки Brent с поставкой в октябре опускаются на 0,6% до $70,88 за баррель, а Light – на 0,75% до $68,38 за баррель. Пара EUR/USD находится на отметке 1,173 (+0,03%), американский доллар стоит 73,48 рубля (-0,15%), тогда как евро – 86,23 рубля (-0,18%). "Роснефть" представила свои финансовые результаты по МСФО за 2 квартал 2021 года. Чистая прибыль компании увеличилась в 6 раз и составила 263 млрд рублей, а прибыль за 1 полугодие составила 431 млрд рублей, против убытка в 98 млрд рубле

In [21]:
for item in news:
    print('start news', news.index(item), end='\r')
    resp = requests.get(item['url'])
    soup = bs(resp.text)
    text_parts = soup.find('div', {'class': 'handmade mid f-newsitem-text'}).find_all('p')
    article = []
    for part in text_parts:
        article.append(part.text) 
    article = ' '.join(article)
    item.update({'article':article})

In [22]:
rosneft = pd.DataFrame(news)

In [23]:
rosneft

,preview_date,preview,url,company,article_time,article_date,article
0,13.08.2021,"Чистая прибыль ""Роснефти"" по РСБУ за 1 полугод...",https://www.finam.ru/analysis/newsitem/chistay...,rosneft,20:55,13.08.2021,"Чистая прибыль ""Роснефти"" по РСБУ за 1 полугод..."
1,13.08.2021,"Совет директоров ""Роснефти"" в ближайшее время ...",https://www.finam.ru/analysis/newsitem/sovet-d...,rosneft,19:44,13.08.2021,"МОСКВА, 13 авг - РИА Новости/Прайм. Совет дире..."
2,13.08.2021,Индексу МосБиржи не удается взять штурмом уров...,https://www.finam.ru/analysis/newsitem/indeksu...,rosneft,15:11,13.08.2021,Российские биржи в пятницу перешли к снижению ...
3,13.08.2021,"""Роснефть"" на конец 2 квартала строит 6 скважи...",https://www.finam.ru/analysis/newsitem/rosneft...,rosneft,10:57,13.08.2021,"""Роснефть"" ведет активную разработку масштабно..."
4,13.08.2021,"""Роснефть"" увеличила добычу нефти и конденсата...",https://www.finam.ru/analysis/newsitem/rosneft...,rosneft,10:50,13.08.2021,"Добыча жидких углеводородов ""Роснефтью"" во 2 к..."
...,...,...,...,...,...,...,...
2438,27.01.2012,"""Газпром"" опустился в рейтинге публичных энерг...",https://www.finam.ru/analysis/newsitem6352F/,rosneft,13:12,27.01.2012,Консалтинговая компания PFC Energy опубликовал...
2439,26.01.2012,"""Роснефть"" будет участвовать в развитии ХК ЦСК...",https://www.finam.ru/analysis/newsitem63489/,rosneft,17:55,26.01.2012,"""Роснефть"" в полном объеме обеспечивает ПХК ЦС..."
2440,23.01.2012,FINAM расширяет возможности маржинальной торговли,https://www.finam.ru/analysis/newsitem631B9/,rosneft,13:15,23.01.2012,Международный брокер FINAM (входит в состав ин...
2441,23.01.2012,"Доказанные запасы углеводородов ""Роснефти"" на ...",https://www.finam.ru/analysis/newsitem631AD/,rosneft,12:22,23.01.2012,"Доказанные запасы углеводородов НК ""Роснефть"" ..."


In [28]:
rosneft.to_csv('rosneft.csv')

CPU times: user 130 ms, sys: 11.1 ms, total: 141 ms
Wall time: 148 ms


In [27]:
rosneft.to_excel('rosneft.xlsx')